# Sprint 機械学習フロー
## 1.このSprintについて
### Sprintの目的
機械学習の実践的な流れを知る  
高い汎化性能を持つモデルを完成させる  
### どのように学ぶか
丁寧な検証が行える状態にした上で、他者の解法を参考に汎化性能の高いモデル作りを進めます。

## 2.機械学習フロー
Kaggleの Home Credit Default Risk コンペティションを題材に、機械学習の実践的な流れを学びます。特に適切な 検証 を行い、高い 汎化性能 のあるモデルを完成させることを目指します。

[Home Credit Default Risk | Kaggle](https://www.kaggle.com/c/home-credit-default-risk)

### 【問題1】クロスバリデーション
事前学習期間では検証用データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。**（ホールドアウト法）**しかし、分割の仕方により精度は変化します。実践的には **クロスバリデーション（交差検証）** を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。

事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。

[sklearn.model_selection.KFold — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)

In [1]:
# ライブラリー取得
import os # ファイルの読み込み先のpath指定する時に必要
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split #データを訓練用データと検証用データに分割
from sklearn.model_selection import KFold # クロスバリデーションのモデルをインポート

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, KFold, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier

In [2]:
# データ取得
df = pd.read_csv("../../dic_ml_ans/application_train.csv")# 返済能力データ
df.head() # 頭の数件を表示させる

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# データの取得
df_train = pd.read_csv("../../dic_ml_ans/application_train.csv")
df_test = pd.read_csv("../../dic_ml_ans/application_test.csv")

# データの表示
display(df_train.head())
display(df_test.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
"""
欠損値を含む列を削除
"""
# 欠損値削除
df_train_drop = df_train.dropna(axis=1)#dropnaで欠損値を削除
df_test_drop =df_test.dropna(axis=1)

In [5]:
"""
機械学習をするには数値である必要があるので文字タイプの列を削除
"""
# 文字(object)列タイプの列を削除
object_colums = df_train_drop.select_dtypes(include=object).columns# 文字型の列を抽出
df_train_drop = df_train_drop.drop(object_colums, axis=1)#文字型の列を削除
df_test_drop = df_test_drop.drop(object_colums, axis=1)#文字型の列を削除

In [6]:
# 訓練用データとテストデータに分割
X = df_train_drop.drop("TARGET", axis=1)# ラベルのTARGETをdrop
y = df_train_drop["TARGET"]#ラベルだけ抽出

In [7]:
# X_testの列に合わせる？？
X_test = df_test_drop[X.columns]

In [8]:
# データを分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=0)

In [9]:
def evaluate(y_test, y_pred):
    """
    2値分類の評価指標を計算する
    """
    acc = metrics.accurach_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, ypred)
    return acc, precision, recall, f1

In [10]:
# クロスバリデーションの実施
SKF = StratifiedKFold(n_splits=3)# 折り畳みの数の指定。少なくとも２以上にしよう。デフォルトは３。
for train_index, test_index in SKF.split(X, y):
    print("TRAIN:", train_index, "TEST:", train_index)

TRAIN: [101986 101997 102002 ... 307508 307509 307510] TEST: [101986 101997 102002 ... 307508 307509 307510]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [     0      1      2 ... 307508 307509 307510]
TRAIN: [     0      1      2 ... 205109 205110 205111] TEST: [     0      1      2 ... 205109 205110 205111]


In [11]:
# ランダムフォレストで推定
forest = RandomForestClassifier(n_estimators=10)
forest.fit(X_train, y_train)
result = cross_val_score(forest, X, y, cv = SKF , scoring = "roc_auc")
result.mean()

0.5389456205280767

In [ ]:
result.predict()

### 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを **パラメータチューニング** と呼びます。パラメータチューニングをある程度自動化する単純な方法としては **グリッドサーチ** があります。

scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。

[sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

In [14]:
#  グリッドサーチに必要なパラメータを設定
grid_param = {"n_estimators": [10, 20, 30], "criterion": ["gini", "entropy"]}
# グリッドサーチモデルにパラメータを設定してインスタンス化
gscv = GridSearchCV(forest, param_grid=grid_param, cv =3, scoring="roc_auc",
                    return_train_score=True)

In [15]:
# 学習の実施
gscv.fit(X, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [20]:
# 結果をデータフレームとして保存
gscv_result = pd.DataFrame.from_dict(gscv.cv_results_)
# 最も良いパラメータを保存
best_params = gscv.best_params_
# ベストスコアを保存
gscv.best_score_
#ベストスコアを出したモデルを保存
best_forest = gscv.best_estimator_

In [40]:
gscv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,5.923373,0.046774,0.350842,0.093684,10,{'n_estimators': 10},0.532510,0.559756,0.547191,0.546486,0.011134,3,0.999809,0.999816,0.999811,0.999812,2.962102e-06
1,52.887893,2.189173,2.866623,0.988114,100,{'n_estimators': 100},0.533083,0.613573,0.595340,0.580665,0.034459,2,1.000000,1.000000,1.000000,1.000000,6.409876e-17
2,172.537446,5.836455,8.530743,2.469106,300,{'n_estimators': 300},0.536907,0.628147,0.607743,0.590933,0.039099,1,1.000000,1.000000,1.000000,1.000000,6.409876e-17


In [52]:
print("ランダムフォレストの木の数：{}".format(best_params))
print("ベストスコア：{}".format(gscv.best_score_))
print("もっとも良いパラメータ：{}".format(best_forest))
print("推定結果：{}".format(y_pred))

ランダムフォレストの木の数：{'n_estimators': 300}
ベストスコア：0.5909325018719722
もっとも良いパラメータ：RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
推定結果：[[0.84       0.16      ]
 [0.89       0.11      ]
 [0.91       0.09      ]
 ...
 [0.80666667 0.19333333]
 [0.84666667 0.15333333]
 [0.67666667 0.32333333]]


### 【問題3】Kernelからの調査
KaggleのKernelから様々なアイデアを見つけ出して、列挙してください。

>LightGBMのモデルを使用

### 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。

その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

In [26]:
grid_param = {"n_estimators": [10, 100, 300]}# 問２のグリッドサーチで出力されたパラメータ（木の数）で実施
gscv = GridSearchCV(forest, param_grid=grid_param, cv=3, scoring="roc_auc",
                    return_train_score=True)

In [27]:
gscv.fit(X, y)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 100, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [31]:
gscv_result = pd.DataFrame.from_dict(gscv.cv_results_)
best_params = gscv.best_params_
gscv.best_score_
best_forest = gscv.best_estimator_

In [32]:
y_pred = best_forest.predict_proba(X_test)

In [55]:
print("ランダムフォレストの木の数：{}".format(best_params))
print("ベストスコア：{}".format(gscv.best_score_))
print("もっとも良いパラメータ：{}".format(best_forest))
print("推定結果：{}".format(y_pred))

ランダムフォレストの木の数：{'n_estimators': 300}
ベストスコア：0.5909325018719722
もっとも良いパラメータ：RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
推定結果：[[0.84       0.16      ]
 [0.89       0.11      ]
 [0.91       0.09      ]
 ...
 [0.80666667 0.19333333]
 [0.84666667 0.15333333]
 [0.67666667 0.32333333]]


In [ ]:
# グリッドサーチから出力されたパラメータを使用したが変わらないように見える。。

### 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

In [33]:
best_RF = RandomForestClassifier(n_estimators=300,
                                criterion="gini",
                                max_depth=8,
                                max_leaf_nodes=None)

In [35]:
best_RF.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [36]:
y_pred_best = best_RF.predict_proba(X_test)

In [39]:
df = pd.DataFrame(y_pred_best)
df

,0,1
0,0.912911,0.087089
1,0.938622,0.061378
2,0.944914,0.055086
3,0.928885,0.071115
4,0.895644,0.104356
5,0.948665,0.051335
6,0.920868,0.079132
7,0.896347,0.103653
8,0.906097,0.093903
9,0.905833,0.094167
